# Input job parameter

In [7]:
import json 

job_para = json.load(open('data/pat_job_test.json'))
job_para

{'job_guid': '80a1ca07-41f9-4f34-b7bb-d01b477bc78d',
 'job_name': 'PAT_Test_AIG',
 'type_of_rating': 'PSOLD',
 'coverage': 'Building + Contents + Time Element',
 'peril_subline': 'All_Perils',
 'subject_premium': 100000000.0,
 'loss_alae_ratio': 1,
 'average_accident_date': '1/1/2022',
 'trend_factor': 1.035,
 'additional_coverage': 2,
 'deductible_treatment': 'Retains Limit',
 'server': 'DFWCAT-RMS5SQL1',
 'edm_database': 'RMS_EPL_AIG_PAT_Testing_202107_EDM181',
 'rdm_database': 'RMS_EPL_AIG_PAT_Testing_202107_RDM181',
 'portinfoid': 6,
 'perilid': 5,
 'analysisid': 5,
 'user_name': 'cxiao',
 'user_ema': 'chengyou.xiao@guycarp.com'}

# Create job

In [ ]:
import numpy as np
import pandas as pd

from pat import PatAnalysis, PatFlag

job = PatAnalysis(job_para)
job.conn_str

# Extract and check data

In [ ]:
job.extract_edm_rdm()
job.check_data()

# Net of Fac

In [ ]:
# Splitting PsuedoPolicies by FAC Layering
if len(job.df_fac) > 0:
    job.net_of_fac()

# Allocate with PSOLD

In [ ]:
job.allocate_with_psold()

# Comparison

In [ ]:
df1 = job.df_pat.reset_index(drop=True)

df2 =pd.read_csv(r'C:\_Working\PAT_20201019\__temp\dfpat.csv',dtype={
                     'LocationIDStack': str
                 }, index_col=0).reset_index(drop=True)
df2.PseudoPolicyID = df2.PseudoPolicyID.str.replace(' _ ', '_').reset_index(drop=True) 
df1.columns = df2.columns


# df1 = df1.sort_values(by=['PseudoPolicyID','LocationIDStack','PseudoLayerID']).reset_index(drop=True)
# df2 = df2.sort_values(by=['PseudoPolicyID','LocationIDStack','PseudoLayerID']).reset_index(drop=True)
# df1 = df1.sort_values(by=['PseudoPolicyID']).reset_index(drop=True)
# df2 = df2.sort_values(by=['PseudoPolicyID']).reset_index(drop=True)
# df2 = df2.sort_values(by=['PseudoPolicyID','FacAttachment', 'FacLimit','FacCeded']).reset_index(drop=True)

# df1 = df1.sort_values(by=['PseudoPolicyID','Retention']).reset_index(drop=True)
# df2 = df2.sort_values(by=['PseudoPolicyID','Retention']).reset_index(drop=True)


print(df1.shape, df2.shape)
pd.DataFrame(df1.dtypes,columns=['DF1']).join(pd.DataFrame(df2.dtypes,columns=['DF2']),how='outer')

In [ ]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

cmp =pd.DataFrame(columns=['name','type', 'match','df1_nan','df2_nan'])
for c in df1.columns:
    if is_string_dtype(df1[c]):
        if np.all(df1[c].fillna(value="") != df2[c].fillna(value="")):
            cmp = cmp.append({'name': c,'type':df1[c].dtype,'match':'NO'}, ignore_index=True)
        else:
            cmp = cmp.append({'name': c,'type':df1[c].dtype,'match':'YES'}, ignore_index=True)
    elif is_numeric_dtype(df1[c]):
        dif =np.max(np.abs(df1[c].astype('float') - df2[c].astype('float')) / np.maximum(df1[c].astype('float'), df2[c].astype('float')) )
        cmp = cmp.append({'name': c,'type':df1[c].dtype,'match':dif,
            'df1_nan':np.sum(np.isnan(df1[c])),
            'df2_nan':np.sum(np.isnan(df2[c]))
        }, ignore_index=True)
    else:
        cmp = cmp.append({'name': c,'type':df1[c].dtype,'match':'UNK'}, ignore_index=True)

cmp

# Test Area

In [ ]:
job.get_validation_counts()

# df_a, df_b, df_c =job.get_validation_data()
# print(df_a.shape, df_b.shape, df_c.shape)

# df_b.sort_values(['LocationIDStack', 'AOI']).to_csv(r'C:\_Working\PAT_20201019\__temp\loc_correction.csv',index=False)

In [6]:
import requests
import json

files = {'para': ('pat_para', open('data/pat_job_test.json'))}
ret = requests.post("http://localhost:5000/api/Jobs", files = files)
ret.status_code, ret.content

(200, b'Analysis submitted!')